# Amplitude do gradiente direcional da anomalia de campo total

Este código utiliza a biblioteca de modelagem e inversão de dados geofísicos denominada [Fatiando a Terra](http://www.fatiando.org/index.html).

### Coisas para importar

In [1]:
#%matplotlib inline
import numpy as np
from fatiando import mesher, gridder, utils
from fatiando.gravmag import polyprism
from fatiando.vis import mpl

### Pontos onde são calculados os dados

In [2]:
shape = (100, 100) # número de pontos ao longo de x e y
area = [-5000, 5000, -5000, 5000] # x mínimo, x máximo, y mínimo e y máximo da área
xp, yp, zp = gridder.regular(area, shape, z=-100) # grade regular localizada em z = -100 m

xc = 0.5*(area[1] + area[0])
yc = 0.5*(area[3] + area[2])

### Inclinação e a declinação do campo geomagnético local

In [3]:
inc = -20. # graus
dec = -20. # graus
amp = 23500.0 # nT

In [5]:
F = utils.ang2vec(amp, inc, dec) # vetor campo geomagnético local

In [6]:
print F

[ 20751.02220665  -7552.75441382  -8037.47336815]


### Modelo

In [7]:
mpl.close('all')
axis = mpl.figure().gca()
mpl.axis('scaled')
vertices = mpl.draw_polygon(area, axis, xy2ne=True)

In [8]:
print vertices

[[ 3661.08786611 -3896.44351464]
 [ 3687.23849372 -3190.37656904]
 [ 3739.53974895 -1490.58577406]
 [ 3739.53974895     0.        ]
 [ 3765.69037657  1699.79079498]
 [ 3687.23849372  3242.67782427]
 [ 3582.63598326  3844.14225941]
 [ 2144.35146444  4079.49790795]
 [ 1098.32635983  4079.49790795]
 [-1385.9832636   4079.49790795]
 [-2536.61087866  4079.49790795]
 [-3870.29288703  4001.0460251 ]
 [-4262.55230126  3817.9916318 ]
 [-4341.0041841   2928.87029289]
 [-3582.63598326  1830.54393305]
 [-2039.74895397  -496.86192469]
 [ -679.91631799 -1882.84518828]
 [  156.90376569 -3373.43096234]
 [ 1255.23012552 -3948.74476987]
 [ 1333.68200837 -4079.49790795]
 [ 1908.9958159  -3582.63598326]
 [  784.51882845 -2065.89958159]
 [ -549.16317992  -104.60251046]
 [-1438.28451883  1333.68200837]
 [-2850.41841004  2850.41841004]
 [-2379.70711297  3321.12970711]
 [-1072.17573222  3033.47280335]
 [  627.61506276  2955.0209205 ]
 [ 2013.59832636  2876.56903766]
 [ 2144.35146444  2850.41841004]
 [ 2693.51

In [19]:
incs = inc # inclinacao da magnetizacao do corpo (em graus)
decs = 90. # declinacao da magnetizacao do corpo (em graus)
amps = 10. # intensidade da magnetizacao do corpo (em A/m)

In [20]:
model = [mesher.PolygonalPrism(vertices, 10, 1010, {'magnetization': utils.ang2vec(amps, incs, decs)})]

### Anomalia de campo total (em nT)

In [21]:
act = polyprism.tf(xp, yp, zp, model, inc, dec)

### Derivadas espaciais da anomalia de campo total (em nT/m)

In [22]:
delta = 10.

dx_act = (polyprism.tf(xp + delta, yp, zp, model, inc, dec) - \
          polyprism.tf(xp - delta, yp, zp, model, inc, dec))/(2.*delta)

dy_act = (polyprism.tf(xp, yp + delta, zp, model, inc, dec) - \
          polyprism.tf(xp, yp - delta, zp, model, inc, dec))/(2.*delta)

dz_act = (polyprism.tf(xp, yp, zp + delta, model, inc, dec) - \
          polyprism.tf(xp, yp, zp - delta, model, inc, dec))/(2.*delta)

In [23]:
teta = np.deg2rad(45.)

dv_act = np.cos(teta)*dx_act + np.sin(teta)*dy_act

### Amplitude do gradiente horizontal

In [24]:
grad_xy = np.sqrt(dx_act**2 + dy_act**2)

### Amplitude do gradiente

In [25]:
grad = np.sqrt(dx_act**2 + dy_act**2 + dz_act**2)

### Plot das derivadas ao longo dos eixos x, y e z

In [26]:
mpl.close('all')
mpl.figure(figsize=(14,15))

mpl.subplot(3,2,1)
mpl.title("Derivada direcional da anomalia de campo total (nT/m)")
mpl.axis('scaled')
mpl.contourf(yp, xp, dv_act, shape, 15, cmap = mpl.get_cmap('RdBu_r'))
mpl.colorbar()
mpl.polygon(model[0], '--k', xy2ne=True)
mpl.xlabel('y (km)')
mpl.ylabel('x (km)')
mpl.m2km()

mpl.subplot(3,2,2)
mpl.title("Derivada em x da anomalia de campo total (nT/m)")
mpl.axis('scaled')
mpl.contourf(yp, xp, dx_act, shape, 15, cmap = mpl.get_cmap('RdBu_r'))
mpl.colorbar()
mpl.polygon(model[0], '--k', xy2ne=True)
mpl.xlabel('y (km)')
mpl.ylabel('x (km)')
mpl.m2km()

mpl.subplot(3,2,3)
mpl.title("Derivada em y da anomalia de campo total (nT/m)")
mpl.axis('scaled')
mpl.contourf(yp, xp, dy_act, shape, 15, cmap = mpl.get_cmap('RdBu_r'))
mpl.colorbar()
mpl.polygon(model[0], '--k', xy2ne=True)
mpl.xlabel('y (km)')
mpl.ylabel('x (km)')
mpl.m2km()

mpl.subplot(3,2,4)
mpl.title("Derivada em z da anomalia de campo total (nT/m)")
mpl.axis('scaled')
mpl.contourf(yp, xp, dz_act, shape, 15, cmap = mpl.get_cmap('RdBu_r'))
mpl.colorbar()
mpl.polygon(model[0], '--k', xy2ne=True)
mpl.xlabel('y (km)')
mpl.ylabel('x (km)')
mpl.m2km()

mpl.subplot(3,2,5)
mpl.title("Amplitude do gradiente horizontal (nT/m)")
mpl.axis('scaled')
mpl.contourf(yp, xp, grad_xy, shape, 15, cmap = mpl.get_cmap('RdBu_r'))
mpl.colorbar()
mpl.polygon(model[0], '--k', xy2ne=True)
mpl.xlabel('y (km)')
mpl.ylabel('x (km)')
mpl.m2km()

mpl.subplot(3,2,6)
mpl.title("Amplitude do gradiente (nT/m)")
mpl.axis('scaled')
mpl.contourf(yp, xp, grad, shape, 15, cmap = mpl.get_cmap('RdBu_r'))
mpl.colorbar()
mpl.polygon(model[0], '--k', xy2ne=True)
mpl.xlabel('y (km)')
mpl.ylabel('x (km)')
mpl.m2km()

mpl.tight_layout()

mpl.savefig('prismas_mag_derivadas.png', dpi=300)

#mpl.show()

<img src='prismas_mag_derivadas.png', width=1200>